# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [1]:
import datetime

In [2]:
from deap_pre_process import *

In [3]:
eeg1d,eeg2d,labels = get_subject_data(2,"aro")

In [4]:
labels[0]

array([0., 1.])

In [5]:
eeg1d.shape

(2400, 128, 32)

In [6]:
eeg1d,eeg2d,labels = get_subject_data(5,"val")

# Making the dummy network


In [7]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15171058008212375722
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11571530746305894046
physical_device_desc: "device: XLA_CPU device"
]


In [9]:
#Constant :
WINDOW_SIZE = 128    #DO NOT CHANGE
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers (for the paper code)
N_FC_IN = 1024
N_FC_OUT= 1024

#FOR PREDICTIONS
N_LABELS = 2
LEARNING_RATE = 0.0001

In [10]:
#FUNCTION FORM https://github.com/ynulonger/ijcnn/blob/master/cv.py
#FOR SHAPING THE INPUT OF LSTM
def apply_fully_connect(x, x_size, fc_size):
    fc_weight = init_weight([x_size, fc_size])
    fc_bias = init_bias([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))

#FOR READOUT OF THE FINAL LAYER
def apply_readout(x, x_size, readout_size):
    readout_weight = init_weight([x_size, readout_size])
    readout_bias = init_bias([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)

In [11]:
#Resetting tf default graph
tf.reset_default_graph()

#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")

#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])

#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")

#Flattening convolution layer
shape = reduced_data.get_shape().as_list()
final_flat = tf.reshape(reduced_data, [-1, shape[1] * shape[2] * shape[3]])
cnn_out_fuse = final_flat

#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Shaping the RNN input
shape = x_data_rnn.get_shape().as_list()
rnn_in_flat = tf.reshape(x_data_rnn, [-1, shape[2]]) #[batch_size*n_time_step, n_electrode]
rnn_fc_in = apply_fully_connect(rnn_in_flat, shape[2], N_FC_IN) #[batch_size*n_time_step, n_electrode]
lstm_in = tf.reshape(rnn_fc_in, [-1,WINDOW_SIZE,N_FC_IN]) #[batch_size, n_time_step, n_fc_in]

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,lstm_in)

#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')

#Selecting the final output form the layer
rnn_output = output[-1] #[batch, fc_size]

#Shaping the output
lstm_fc_out = dnn_layer(rnn_output,N_FC_OUT) #[batch_size, n_fc_out]
lstm_fc_drop = tf.nn.dropout(lstm_fc_out, keep_prob)

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,N_LABELS)) #[low, high]

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
#Flattening the layer
final_connected_layer = tf.concat([cnn_out_fuse, lstm_fc_drop], axis=1)
shape = final_connected_layer.get_shape().as_list()
print("\nfuse_cnn_rnn:", shape)


fuse_cnn_rnn: [None, 2077]


In [13]:
#Creating output variables 
y_ = apply_readout(final_connected_layer, shape[1], N_LABELS)   #(1,2)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

#Evaluation values
correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(y_), 1), tf.argmax(y_true, 1))

#Making cost functions
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y_true), name='loss')
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
# Saver and variable initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [15]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [16]:
aro_or_val="aro"

In [ ]:
#Training and testing on valance
with tf.Session() as sess:
    sess.run(init)
    try:
        saver.restore(sess, aro_or_val+"_TrainedModel1/model")
        print("[*] Model Restored.")
    except:
        pass
    try:
        for i in range(10):        # num of people
            sub = np.random.randint(1,33)
            print(i+1,"\b.) SUBJECT:",sub)
            eeg1d,eeg2d,labels = get_subject_data(sub,aro_or_val)
            t1 = datetime.datetime.now()
            num_sec=2400                 # for total do 40*60 = 2400 seconds
            print("[*] Training",num_sec,"seconds.")
            for k in range(num_sec):
                print('\r',k,'/',num_sec,end='')
                pred_labels=labels[k].reshape(1,2)
                x_rnn = eeg1d[k].reshape(-1,WINDOW_SIZE,32)  #(1,128,32)
                x_cnn = eeg2d[k]
                sess.run(train, feed_dict = {keep_prob:0.5,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: pred_labels})

            print("\r",num_sec,'/',num_sec)
            t2 = datetime.datetime.now()
            print((t2 -t1).seconds," seconds",(t2 -t1).microseconds, "microseconds")
            if not i%5:
                test_s = np.random.randint(1,33)
                test_eeg1d,test_eeg2d,test_labels = get_subject_data(test_s,aro_or_val)
                print("\n[+] Calculating accuracy")
                pred = []
                tes_sec = 1000
                for j in range(tes_sec):
                    print('\r',j,'/',tes_sec,end='')
                    actual = test_labels[j].reshape(1,2)
                    x_rnn = test_eeg1d[j].reshape(-1,WINDOW_SIZE,32)
                    x_cnn = test_eeg2d[j]
                    ret = sess.run(correct_prediction ,feed_dict = {keep_prob:1,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: actual})
                    pred.append(ret)
                accuracy = tf.reduce_mean(tf.cast(np.array(pred), tf.float32), name='accuracy')
                print("\nAccuracy: ", sess.run(accuracy))
                print("\n")
        saver.save(sess, aro_or_val+"_TrainedModel1/model")
        print("[*] Model Saved.")
    except KeyboardInterrupt:
        saver.save(sess, aro_or_val+"_TrainedModel1/model")
        print("\n[*] Model Saved.")


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from aro_TrainedModel1/model
[*] Model Restored.
1.) SUBJECT: 13
[*] Training 2400 seconds.
 14 / 2400

In [ ]:
with tf.Session() as sess:
        saver.restore(sess, aro_or_val+"_TrainedModel1/model")
        test_s = np.random.randint(1,33)
        test_eeg1d,test_eeg2d,test_labels = get_subject_data(test_s,aro_or_val)
        print("\n[+] Calculating accuracy")
        pred = []
        tes_sec = 1000
        for j in range(tes_sec):
            print('\r',j,'/',tes_sec,end='')
            actual = test_labels[j].reshape(1,2)
            x_rnn = test_eeg1d[j].reshape(-1,WINDOW_SIZE,32)
            x_cnn = test_eeg2d[j]
            ret = sess.run(correct_prediction ,feed_dict = {keep_prob:1,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: actual})
            pred.append(ret)
        accuracy = tf.reduce_mean(tf.cast(np.array(pred), tf.float32), name='accuracy')
        print("\nAccuracy: ", sess.run(accuracy))
        print("\n")